# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4911, uptime 0:00:33
mini_beamline                    RUNNING   pid 4912, uptime 0:00:33
random_walk                      RUNNING   pid 4913, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4914, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4915, uptime 0:00:33
simple                           RUNNING   pid 4916, uptime 0:00:33
thermo_sim                       RUNNING   pid 4917, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4918, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-09 20:08:41
Persistent Unique Scan ID: '16466e07-bfe1-430e-9a4e-54772fab1249'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:08:41.2 |     -1.000 |      94394 |
|         2 | 20:08:41.2 |     -0.500 |      95249 |
|         3 | 20:08:41.2 |      0.000 |      96169 |
|         4 | 20:08:41.2 |      0.500 |      95476 |
|         5 | 20:08:41.3 |      1.000 |      93302 |


+-----------+------------+------------+------------+
generator scan ['16466e07'] (scan num: 3)





('16466e07-bfe1-430e-9a4e-54772fab1249',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/16466e07-bfe1-430e-9a4e-54772fab1249-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-09 20:08:41
Persistent Unique Scan ID: '8918b99f-0065-4c46-b1e7-488e73bdaa00'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:08:41.7 |          -1 | 1053548956 |


|         2 | 20:08:41.8 |          -0 | 1053705953 |


|         3 | 20:08:41.9 |           0 | 1056920113 |


|         4 | 20:08:42.1 |           0 | 1060759209 |


|         5 | 20:08:42.1 |           1 | 1064715371 |
+-----------+------------+-------------+------------+
generator scan ['8918b99f'] (scan num: 4)





('8918b99f-0065-4c46-b1e7-488e73bdaa00',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-09 20:08:42
Persistent Unique Scan ID: 'b892fa1a-09ea-46e4-b4e4-db11f2f3d089'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:08:42.6 | 1086685840 |


|         2 | 20:08:43.5 | 1131407339 |


|         3 | 20:08:44.5 | 1097821183 |


...

|        28 | 20:09:09.6 | 1054032262 |


|        29 | 20:09:10.6 | 1089527238 |


|        30 | 20:09:11.6 | 1131688160 |


+-----------+------------+------------+
generator count ['b892fa1a'] (scan num: 5)





('b892fa1a-09ea-46e4-b4e4-db11f2f3d089',)